---

## 📊 Climate Visualization Pipeline for Nepal  
**Author:** @Siddhant Baral  
**Dataset:** EC-Earth3-Veg (CMIP6, ScenarioMIP)  

This notebook automates the visualization of annual climate variables from EC-Earth3-Veg GCM outputs for Nepal. It processes `.nc` files across two scenarios (`ssp245`, `ssp585`) and three variables (`tasmin`, `tasmax`, `pr`), generating:

- 🗺️ Animated spatial maps of annual means/sums  
- 📈 Trend plots with rolling averages and extremes  
- 🎞️ Combined GIFs for each variable-scenario pair  
- ⚡ Parallel execution with runtime logging and performance tracking  

All outputs are saved to `/content/drive/MyDrive/climate_outputs`.

---

### 📄 License & Attribution

This work uses downscaled results from the **Rhodium Group / Climate Impact Lab Global Downscaled Projections for Climate Impacts Research (R/CIL GDPCIR)** dataset. These results are licensed under the [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/). Please note:

- This license applies **only to the downscaled results**.
- The original GCM output from EC-Earth3-Veg may have **different license terms**.
- Proper attribution is required for both the **original model** and the **R/CIL GDPCIR dataset**.

For full license details, see:
- [EC-Earth3-Veg license text](https://raw.githubusercontent.com/ClimateImpactLab/downscaleCMIP6/master/data_licenses/EC-Earth3-Veg.txt)  
- [R/CIL GDPCIR README and citation info](https://github.com/ClimateImpactLab/downscaleCMIP6/blob/master/README.rst)  
- [Code license for downscaling tools](https://github.com/ClimateImpactLab/downscaleCMIP6/blob/master/LICENSE)

---

### 📚 Citations

**CMIP6 Citation:**  
EC-Earth Consortium (EC-Earth) (2019). *EC-Earth3-Veg model output prepared for CMIP6 CMIP*. Version 20200225. Earth System Grid Federation. [https://doi.org/10.22033/ESGF/CMIP6.642](https://doi.org/10.22033/ESGF/CMIP6.642)

**ScenarioMIP Citation:**  
EC-Earth Consortium (EC-Earth) (2019). *EC-Earth3-Veg model output prepared for CMIP6 ScenarioMIP*. Version 20200225. Earth System Grid Federation. [https://doi.org/10.22033/ESGF/CMIP6.727](https://doi.org/10.22033/ESGF/CMIP6.727)

In [ ]:
# For Nepal
import time
from datetime import timedelta
from concurrent.futures import ProcessPoolExecutor, as_completed

def visualize_climate_variable(file_path, var_name, scenario, label, cmap, unit, is_sum=False, shape=None, save_dir="/content/drive/MyDrive/climate_outputs"):
    import xarray as xr, geopandas as gpd, matplotlib.pyplot as plt
    from matplotlib.animation import FuncAnimation
    from matplotlib.colors import Normalize
    from matplotlib.collections import PolyCollection
    import numpy as np, pandas as pd
    from PIL import Image

    ds = xr.open_dataset(file_path)
    ds["time"] = pd.to_datetime(ds["time"].values)
    ds = ds.set_coords("time")

    annual = ds[var_name].groupby("time.year")
    annual_data = annual.sum(dim="time") if is_sum else annual.mean(dim="time")
    years = annual_data.year.values
    mean_values = annual_data.mean(dim=["lat", "lon"]).values
    rolling = xr.DataArray(mean_values, dims="year", coords={"year": years}).rolling(year=10, center=True).mean()
    max_idx, min_idx = np.nanargmax(mean_values), np.nanargmin(mean_values)

    # === Map Animation ===
    fig, ax = plt.subplots(figsize=(6, 4))
    data0 = annual_data.isel(year=0)
    im = ax.imshow(data0.values,
                   extent=[data0.lon.min(), data0.lon.max(), data0.lat.min(), data0.lat.max()],
                   origin='lower', cmap=cmap,
                   vmin=float(np.nanmin(annual_data)), vmax=float(np.nanmax(annual_data)), aspect='auto')
    shape.boundary.plot(ax=ax, edgecolor="black", linewidth=1.2, zorder=5)
    cbar = plt.colorbar(im, ax=ax, orientation="vertical", fraction=0.035, pad=0.02, shrink=0.8)
    cbar.set_label(f"{label} ({unit})", fontsize=9)
    ax.set_xlim(*shape.total_bounds[[0, 2]] + np.array([-0.2, 0.2]))
    ax.set_ylim(*shape.total_bounds[[1, 3]] + np.array([-0.2, 0.2]))
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    title_text = ax.set_title("")

    def update_map(frame):
        im.set_data(annual_data.isel(year=frame).values)
        title_text.set_text(f"Nepal {label} {scenario} - Year {int(years[frame])}")
        return [im, title_text]

    map_gif = f"{save_dir}/{var_name}_{scenario}_map.gif"
    map_anim = FuncAnimation(fig, update_map, frames=len(years), interval=300)
    map_anim.save(map_gif, writer="pillow")
    plt.close(fig)

    # === Trend Animation ===
    fig, ax = plt.subplots(figsize=(7, 4))
    cmap_trend = plt.cm.viridis
    norm = Normalize(vmin=years.min(), vmax=years.max())

    def update_trend(frame):
        ax.clear()
        for i in range(frame):
            ax.plot(years[i:i+2], mean_values[i:i+2], color=cmap_trend(norm(years[i])), linewidth=2.5)
            verts = [(years[i], 0), (years[i], mean_values[i]), (years[i+1], mean_values[i+1]), (years[i+1], 0)]
            ax.add_collection(PolyCollection([verts], facecolors=[cmap_trend(norm(years[i]))], alpha=0.35))
        ax.plot(years, rolling, color="black", linestyle="--", linewidth=2.5, label="10-yr trend")
        ax.scatter(years[max_idx], mean_values[max_idx], color="blue", marker="^", s=120, label="Max Year")
        ax.scatter(years[min_idx], mean_values[min_idx], color="red", marker="v", s=120, label="Min Year")
        if frame >= max_idx:
            ax.text(years[max_idx], mean_values[max_idx]*1.05, f"{years[max_idx]}", color="blue", fontsize=9,
                    ha="center", va="bottom", fontweight="bold")
        if frame >= min_idx:
            ax.text(years[min_idx], mean_values[min_idx]*0.95, f"{years[min_idx]}", color="red", fontsize=9,
                    ha="center", va="top", fontweight="bold")
        ax.set_xlim(years[0], years[-1])
        ax.set_ylim(np.nanmin(mean_values)*0.95, np.nanmax(mean_values)*1.15)
        ax.set_title(f"{label} in Nepal {scenario} ({unit})", fontsize=12, fontweight="bold")
        ax.set_xlabel("Year")
        ax.set_ylabel(unit)
        ax.grid(True, linestyle="--", alpha=0.6)
        ax.legend(loc="upper left", fontsize=9, frameon=True, facecolor="white", framealpha=0.8)
        fig.subplots_adjust(bottom=0.15, top=0.9)

    trend_gif = f"{save_dir}/{var_name}_{scenario}_trend.gif"
    trend_anim = FuncAnimation(fig, update_trend, frames=len(years), interval=300)
    trend_anim.save(trend_gif, writer="pillow")
    plt.close(fig)

    # === Combine GIFs ===
    map_frames = Image.open(map_gif)
    trend_frames = Image.open(trend_gif)
    frames = []
    for i in range(map_frames.n_frames):
        map_frames.seek(i)
        trend_frames.seek(i)
        map_img = map_frames.convert("RGBA")
        trend_img = trend_frames.convert("RGBA").resize((map_img.width, trend_frames.height))
        new_frame = Image.new("RGBA", (map_img.width, map_img.height + trend_img.height), (255, 255, 255, 255))
        new_frame.paste(map_img, (0, 0))
        new_frame.paste(trend_img, (0, map_img.height))
        frames.append(new_frame)

    combined_gif = f"{save_dir}/{var_name}_{scenario}_combined.gif"
    frames[0].save(combined_gif, save_all=True, append_images=frames[1:], loop=0, duration=300)
    print("✅ Saved:", combined_gif)


# Load Nepal shapefile once
import geopandas as gpd
nepal_shape = gpd.read_file("/content/drive/MyDrive/climate_data/Nepal_shp/dissolved.shp").to_crs("EPSG:4326")

# Define scenarios and variable configs
scenarios = ["ssp245", "ssp585"]
variables = {
    "tasmin": {"label": "Annual Min Temperature", "cmap": "coolwarm", "unit": "K", "is_sum": False},
    "tasmax": {"label": "Annual Max Temperature", "cmap": "coolwarm", "unit": "K", "is_sum": False},
    "pr":     {"label": "Annual Precipitation",    "cmap": "YlGnBu",   "unit": "mm/year", "is_sum": True}
}

# Prepare task arguments
task_args = []
for scenario in scenarios:
    for var, props in variables.items():
        file_path = f"/content/drive/MyDrive/EC-Earth3-Veg/{scenario}/{var}_E3Veg_{scenario}_nepal_2015_2100_reparsed.nc"
        task_args.append((file_path, var, scenario, props, nepal_shape))

# Wrapper with logging
def run_visualization(args):
    file_path, var_name, scenario, props, shape = args
    start = time.time()
    print(f"▶️ Starting: {var_name} | {scenario}")
    visualize_climate_variable(
        file_path=file_path,
        var_name=var_name,
        scenario=scenario,
        label=props["label"],
        cmap=props["cmap"],
        unit=props["unit"],
        is_sum=props["is_sum"],
        shape=shape
    )
    end = time.time()
    duration = timedelta(seconds=end - start)
    print(f"✅ Done: {var_name} | {scenario} in {duration}")
    return (var_name, scenario, duration)

# Run in parallel with progress tracking
start_all = time.time()
results = []
with ProcessPoolExecutor(max_workers=3) as executor:
    futures = {executor.submit(run_visualization, args): args for args in task_args}
    total = len(futures)
    completed = 0
    for future in as_completed(futures):
        result = future.result()
        results.append(result)
        completed += 1
        elapsed = time.time() - start_all
        avg_time = elapsed / completed
        remaining = avg_time * (total - completed)
        print(f"⏳ {completed}/{total} done | Est. time left: {timedelta(seconds=remaining)}")
# Summary
print("\n📋 Task Summary:")
for var, scenario, duration in results:
    print(f"• {var} | {scenario} → {duration}")
print(f"\n🏁 Total time: {timedelta(seconds=time.time() - start_all)}")


▶️ Starting: tasmin | ssp245
▶️ Starting: tasmax | ssp245
▶️ Starting: pr | ssp245
✅ Saved: /content/drive/MyDrive/climate_outputs/tasmin_ssp245_combined.gif
✅ Done: tasmin | ssp245 in 0:02:26.306591
▶️ Starting: tasmin | ssp585
⏳ 1/6 done | Est. time left: 0:12:11.861461
✅ Saved: /content/drive/MyDrive/climate_outputs/pr_ssp245_combined.gif
✅ Done: pr | ssp245 in 0:02:27.338528
▶️ Starting: tasmax | ssp585
⏳ 2/6 done | Est. time left: 0:04:54.994733
✅ Saved: /content/drive/MyDrive/climate_outputs/tasmax_ssp245_combined.gif
✅ Done: tasmax | ssp245 in 0:02:29.126221
▶️ Starting: pr | ssp585
⏳ 3/6 done | Est. time left: 0:02:29.198211
✅ Saved: /content/drive/MyDrive/climate_outputs/tasmin_ssp585_combined.gif
✅ Done: tasmin | ssp585 in 0:02:17.036085
⏳ 4/6 done | Est. time left: 0:02:21.719863
✅ Saved: /content/drive/MyDrive/climate_outputs/tasmax_ssp585_combined.gif
✅ Done: tasmax | ssp585 in 0:02:20.320236
⏳ 5/6 done | Est. time left: 0:00:57.563320
✅ Saved: /content/drive/MyDrive/clima

In [ ]:
# For West Rapti River Basin
import os
import time
from datetime import timedelta
from concurrent.futures import ProcessPoolExecutor, as_completed

def visualize_climate_variable(file_path, var_name, scenario, label, cmap, unit, is_sum=False, shape=None, save_dir="/content/drive/MyDrive/climate_outputs/wrb_maps"):
    import xarray as xr, geopandas as gpd, matplotlib.pyplot as plt
    from matplotlib.animation import FuncAnimation
    from matplotlib.colors import Normalize
    from matplotlib.collections import PolyCollection
    import numpy as np, pandas as pd
    from PIL import Image

    os.makedirs(save_dir, exist_ok=True)

    ds = xr.open_dataset(file_path)
    ds["time"] = pd.to_datetime(ds["time"].values)
    ds = ds.set_coords("time")

    annual = ds[var_name].groupby("time.year")
    annual_data = annual.sum(dim="time") if is_sum else annual.mean(dim="time")
    years = annual_data.year.values
    mean_values = annual_data.mean(dim=["lat", "lon"]).values
    rolling = xr.DataArray(mean_values, dims="year", coords={"year": years}).rolling(year=10, center=True).mean()
    max_idx, min_idx = np.nanargmax(mean_values), np.nanargmin(mean_values)

    # === Map Animation ===
    fig, ax = plt.subplots(figsize=(6, 4))
    data0 = annual_data.isel(year=0)
    im = ax.imshow(data0.values,
                   extent=[data0.lon.min(), data0.lon.max(), data0.lat.min(), data0.lat.max()],
                   origin='lower', cmap=cmap,
                   vmin=float(np.nanmin(annual_data)), vmax=float(np.nanmax(annual_data)), aspect='auto')
    shape.boundary.plot(ax=ax, edgecolor="black", linewidth=1.2, zorder=5)
    cbar = plt.colorbar(im, ax=ax, orientation="vertical", fraction=0.035, pad=0.02, shrink=0.8)
    cbar.set_label(f"{label} ({unit})", fontsize=9)
    ax.set_xlim(*shape.total_bounds[[0, 2]] + np.array([-0.2, 0.2]))
    ax.set_ylim(*shape.total_bounds[[1, 3]] + np.array([-0.2, 0.2]))
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    title_text = ax.set_title("")

    def update_map(frame):
        im.set_data(annual_data.isel(year=frame).values)
        title_text.set_text(f"WRB {label} {scenario} - Year {int(years[frame])}")
        return [im, title_text]

    map_gif = f"{save_dir}/{var_name}_{scenario}_map.gif"
    map_anim = FuncAnimation(fig, update_map, frames=len(years), interval=300)
    map_anim.save(map_gif, writer="pillow")
    plt.close(fig)

    # === Trend Animation ===
    fig, ax = plt.subplots(figsize=(7, 4))
    cmap_trend = plt.cm.viridis
    norm = Normalize(vmin=years.min(), vmax=years.max())

    def update_trend(frame):
        ax.clear()
        for i in range(frame):
            ax.plot(years[i:i+2], mean_values[i:i+2], color=cmap_trend(norm(years[i])), linewidth=2.5)
            verts = [(years[i], 0), (years[i], mean_values[i]), (years[i+1], mean_values[i+1]), (years[i+1], 0)]
            ax.add_collection(PolyCollection([verts], facecolors=[cmap_trend(norm(years[i]))], alpha=0.35))
        ax.plot(years, rolling, color="black", linestyle="--", linewidth=2.5, label="10-yr trend")
        ax.scatter(years[max_idx], mean_values[max_idx], color="blue", marker="^", s=120, label="Max Year")
        ax.scatter(years[min_idx], mean_values[min_idx], color="red", marker="v", s=120, label="Min Year")
        if frame >= max_idx:
            ax.text(years[max_idx], mean_values[max_idx]*1.05, f"{years[max_idx]}", color="blue", fontsize=9,
                    ha="center", va="bottom", fontweight="bold")
        if frame >= min_idx:
            ax.text(years[min_idx], mean_values[min_idx]*0.95, f"{years[min_idx]}", color="red", fontsize=9,
                    ha="center", va="top", fontweight="bold")
        ax.set_xlim(years[0], years[-1])
        ax.set_ylim(np.nanmin(mean_values)*0.95, np.nanmax(mean_values)*1.15)
        ax.set_title(f"{label} in WRB {scenario} ({unit})", fontsize=12, fontweight="bold")
        ax.set_xlabel("Year")
        ax.set_ylabel(unit)
        ax.grid(True, linestyle="--", alpha=0.6)
        ax.legend(loc="upper left", fontsize=9, frameon=True, facecolor="white", framealpha=0.8)
        fig.subplots_adjust(bottom=0.15, top=0.9)

    trend_gif = f"{save_dir}/{var_name}_{scenario}_trend.gif"
    trend_anim = FuncAnimation(fig, update_trend, frames=len(years), interval=300)
    trend_anim.save(trend_gif, writer="pillow")
    plt.close(fig)

    # === Combine GIFs ===
    map_frames = Image.open(map_gif)
    trend_frames = Image.open(trend_gif)
    frames = []
    for i in range(map_frames.n_frames):
        map_frames.seek(i)
        trend_frames.seek(i)
        map_img = map_frames.convert("RGBA")
        trend_img = trend_frames.convert("RGBA").resize((map_img.width, trend_frames.height))
        new_frame = Image.new("RGBA", (map_img.width, map_img.height + trend_img.height), (255, 255, 255, 255))
        new_frame.paste(map_img, (0, 0))
        new_frame.paste(trend_img, (0, map_img.height))
        frames.append(new_frame)

    combined_gif = f"{save_dir}/{var_name}_{scenario}_combined.gif"
    frames[0].save(combined_gif, save_all=True, append_images=frames[1:], loop=0, duration=300)
    print("✅ Saved:", combined_gif)


# === Load WRB Shapefile ===
import geopandas as gpd
wrb_shape = gpd.read_file("/content/drive/MyDrive/WRB/watershed/watershed.shp").to_crs("EPSG:4326")

# === Define Scenarios and Variables ===
scenarios = ["ssp245", "ssp585"]
variables = {
    "tasmin": {"label": "Annual Min Temperature", "cmap": "coolwarm", "unit": "K", "is_sum": False},
    "tasmax": {"label": "Annual Max Temperature", "cmap": "coolwarm", "unit": "K", "is_sum": False},
    "pr":     {"label": "Annual Precipitation",    "cmap": "YlGnBu",   "unit": "mm/year", "is_sum": True}
}

# === Prepare Task Arguments ===
task_args = []
for scenario in scenarios:
    for var, props in variables.items():
        file_path = f"/content/drive/MyDrive/WRB/gcm_data/{scenario}/wrb_{var}_{scenario}_fut.nc"
        task_args.append((file_path, var, scenario, props, wrb_shape))

# === Run in Parallel ===
def run_visualization(args):
    file_path, var_name, scenario, props, shape = args
    start = time.time()
    print(f"▶️ Starting: {var_name} | {scenario}")
    visualize_climate_variable(
        file_path=file_path,
        var_name=var_name,
        scenario=scenario,
        label=props["label"],
        cmap=props["cmap"],
        unit=props["unit"],
        is_sum=props["is_sum"],
        shape=shape
    )
    end = time.time()
    duration = timedelta(seconds=end - start)
    print(f"✅ Done: {var_name} | {scenario} in {duration}")
    return (var_name, scenario, duration)

start_all = time.time()
results = []
with ProcessPoolExecutor(max_workers=3) as executor:
    futures = {executor.submit(run_visualization, args): args for args in task_args}
    total = len(futures)
    completed = 0
    for future in as_completed(futures):
        result = future.result()
        results.append(result)
        completed += 1
        elapsed = time.time() - start_all
        avg_time = elapsed / completed
        remaining = avg_time * (total - completed)
        print(f"⏳ {completed}/{total} done | Est. time left: {timedelta(seconds=remaining)}")
# Summary
print("\n📋 Task Summary:")
for var, scenario, duration in results:
    print(f"• {var} | {scenario} → {duration}")
print(f"\n🏁 Total time: {timedelta(seconds=time.time() - start_all)}")

▶️ Starting: tasmax | ssp245▶️ Starting: pr | ssp245
▶️ Starting: tasmin | ssp245

✅ Saved: /content/drive/MyDrive/climate_outputs/wrb_maps/tasmin_ssp245_combined.gif
✅ Done: tasmin | ssp245 in 0:02:05.734621
▶️ Starting: tasmin | ssp585
⏳ 1/6 done | Est. time left: 0:10:28.949674
✅ Saved: /content/drive/MyDrive/climate_outputs/wrb_maps/pr_ssp245_combined.gif
✅ Done: pr | ssp245 in 0:02:09.077653
▶️ Starting: tasmax | ssp585
⏳ 2/6 done | Est. time left: 0:04:18.286958
✅ Saved: /content/drive/MyDrive/climate_outputs/wrb_maps/tasmax_ssp245_combined.gif
✅ Done: tasmax | ssp245 in 0:02:10.366982
▶️ Starting: pr | ssp585
⏳ 3/6 done | Est. time left: 0:02:10.422635
✅ Saved: /content/drive/MyDrive/climate_outputs/wrb_maps/tasmin_ssp585_combined.gif
✅ Done: tasmin | ssp585 in 0:02:06.395281
⏳ 4/6 done | Est. time left: 0:02:06.107218
✅ Saved: /content/drive/MyDrive/climate_outputs/wrb_maps/tasmax_ssp585_combined.gif
✅ Done: tasmax | ssp585 in 0:02:06.622111
⏳ 5/6 done | Est. time left: 0:00:51

In [ ]:
# Even tighter WRB
import os
import time
from datetime import timedelta
from concurrent.futures import ProcessPoolExecutor, as_completed
import geopandas as gpd

# === Load WRB Shapefile (for boundary overlay only)
wrb_shape = gpd.read_file("/content/drive/MyDrive/WRB/watershed/watershed.shp").to_crs("EPSG:4326")

# === Visualization Function
def visualize_climate_variable(file_path, var_name, scenario, label, cmap, unit, is_sum=False, shape=None, save_dir="/content/drive/MyDrive/climate_outputs/wrb_gifs/20kmbuffer"):
    import xarray as xr, matplotlib.pyplot as plt
    from matplotlib.animation import FuncAnimation
    from matplotlib.colors import Normalize
    from matplotlib.collections import PolyCollection
    import numpy as np, pandas as pd
    from PIL import Image

    os.makedirs(save_dir, exist_ok=True)

    ds = xr.open_dataset(file_path)
    ds["time"] = pd.to_datetime(ds["time"].values)
    ds = ds.set_coords("time")

    annual = ds[var_name].groupby("time.year")
    annual_data = annual.sum(dim="time") if is_sum else annual.mean(dim="time")
    years = annual_data.year.values
    mean_values = annual_data.mean(dim=["lat", "lon"]).values
    rolling = xr.DataArray(mean_values, dims="year", coords={"year": years}).rolling(year=10, center=True).mean()
    max_idx, min_idx = np.nanargmax(mean_values), np.nanargmin(mean_values)

    # === Map Animation ===
    fig, ax = plt.subplots(figsize=(6, 4))
    data0 = annual_data.isel(year=0)
    im = ax.imshow(data0.values,
                   extent=[data0.lon.min(), data0.lon.max(), data0.lat.min(), data0.lat.max()],
                   origin='lower', cmap=cmap,
                   vmin=float(np.nanmin(annual_data)), vmax=float(np.nanmax(annual_data)), aspect='auto')
    shape.boundary.plot(ax=ax, edgecolor="black", linewidth=1.2, zorder=5)
    cbar = plt.colorbar(im, ax=ax, orientation="vertical", fraction=0.035, pad=0.02, shrink=0.8)
    cbar.set_label(f"{label} ({unit})", fontsize=9)
    ax.set_xlim(*shape.total_bounds[[0, 2]] + np.array([-0.2, 0.2]))
    ax.set_ylim(*shape.total_bounds[[1, 3]] + np.array([-0.2, 0.2]))
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    title_text = ax.set_title("")

    def update_map(frame):
        im.set_data(annual_data.isel(year=frame).values)
        title_text.set_text(f"WRB {label} {scenario} - Year {int(years[frame])}")
        return [im, title_text]

    map_gif = f"{save_dir}/{var_name}_{scenario}_map.gif"
    map_anim = FuncAnimation(fig, update_map, frames=len(years), interval=300)
    map_anim.save(map_gif, writer="pillow")
    plt.close(fig)

    # === Trend Animation ===
    fig, ax = plt.subplots(figsize=(7, 4))
    cmap_trend = plt.cm.viridis
    norm = Normalize(vmin=years.min(), vmax=years.max())

    def update_trend(frame):
        ax.clear()
        for i in range(frame):
            ax.plot(years[i:i+2], mean_values[i:i+2], color=cmap_trend(norm(years[i])), linewidth=2.5)
            verts = [(years[i], 0), (years[i], mean_values[i]), (years[i+1], mean_values[i+1]), (years[i+1], 0)]
            ax.add_collection(PolyCollection([verts], facecolors=[cmap_trend(norm(years[i]))], alpha=0.35))
        ax.plot(years, rolling, color="black", linestyle="--", linewidth=2.5, label="10-yr trend")
        ax.scatter(years[max_idx], mean_values[max_idx], color="blue", marker="^", s=120, label="Max Year")
        ax.scatter(years[min_idx], mean_values[min_idx], color="red", marker="v", s=120, label="Min Year")
        if frame >= max_idx:
            ax.text(years[max_idx], mean_values[max_idx]*1.05, f"{years[max_idx]}", color="blue", fontsize=9,
                    ha="center", va="bottom", fontweight="bold")
        if frame >= min_idx:
            ax.text(years[min_idx], mean_values[min_idx]*0.95, f"{years[min_idx]}", color="red", fontsize=9,
                    ha="center", va="top", fontweight="bold")
        ax.set_xlim(years[0], years[-1])
        ax.set_ylim(np.nanmin(mean_values)*0.95, np.nanmax(mean_values)*1.15)
        ax.set_title(f"{label} in WRB {scenario} ({unit})", fontsize=12, fontweight="bold")
        ax.set_xlabel("Year")
        ax.set_ylabel(unit)
        ax.grid(True, linestyle="--", alpha=0.6)
        ax.legend(loc="upper left", fontsize=9, frameon=True, facecolor="white", framealpha=0.8)
        fig.subplots_adjust(bottom=0.15, top=0.9)

    trend_gif = f"{save_dir}/{var_name}_{scenario}_trend.gif"
    trend_anim = FuncAnimation(fig, update_trend, frames=len(years), interval=300)
    trend_anim.save(trend_gif, writer="pillow")
    plt.close(fig)

    # === Combine GIFs ===
    map_frames = Image.open(map_gif)
    trend_frames = Image.open(trend_gif)
    frames = []
    for i in range(map_frames.n_frames):
        map_frames.seek(i)
        trend_frames.seek(i)
        map_img = map_frames.convert("RGBA")
        trend_img = trend_frames.convert("RGBA").resize((map_img.width, trend_frames.height))
        new_frame = Image.new("RGBA", (map_img.width, map_img.height + trend_img.height), (255, 255, 255, 255))
        new_frame.paste(map_img, (0, 0))
        new_frame.paste(trend_img, (0, map_img.height))
        frames.append(new_frame)

    combined_gif = f"{save_dir}/{var_name}_{scenario}_combined.gif"
    frames[0].save(combined_gif, save_all=True, append_images=frames[1:], loop=0, duration=300)
    print("✅ Saved:", combined_gif)

# === Define Scenarios and Variables
scenarios = ["ssp245", "ssp585"]
variables = {
    "tasmin": {"label": "Annual Min Temperature", "cmap": "coolwarm", "unit": "K", "is_sum": False},
    "tasmax": {"label": "Annual Max Temperature", "cmap": "coolwarm", "unit": "K", "is_sum": False},
    "pr":     {"label": "Annual Precipitation",    "cmap": "YlGnBu",   "unit": "mm/year", "is_sum": True}
}

# === Prepare Task Arguments (using clipped files)
task_args = []
for scenario in scenarios:
    for var, props in variables.items():
        file_path = f"/content/drive/MyDrive/WRB/clipped_data/20km/wrb_{var}_{scenario}_fut_clipped.nc"
        task_args.append((file_path, var, scenario, props, wrb_shape))

# === Run in Parallel
def run_visualization(args):
    file_path, var_name, scenario, props, shape = args
    start = time.time()
    print(f"▶️ Starting: {var_name} | {scenario}")
    visualize_climate_variable(
        file_path=file_path,
        var_name=var_name,
        scenario=scenario,
        label=props["label"],
        cmap=props["cmap"],
        unit=props["unit"],
        is_sum=props["is_sum"],
        shape=shape
    )
    end = time.time()
    duration = timedelta(seconds=end - start)
    print(f"✅ Done: {var_name} | {scenario} in {duration}")
    return (var_name, scenario, duration)

start_all = time.time()
results = []
with ProcessPoolExecutor(max_workers=3) as executor:
    futures = {executor.submit(run_visualization, args): args for args in task_args}
    total = len(futures)
    completed = 0
    for future in as_completed(futures):
        result = future.result()
        results.append(result)
        completed += 1
        elapsed = time.time() - start_all
        avg_time = elapsed / completed
        remaining = avg_time * (total - completed)
        print(f"⏳ {completed}/{total} done | Est. time left: {timedelta(seconds=remaining)}")
# Summary
print("\n📋 Task Summary:")
for var, scenario, duration in results:
    print(f"• {var} | {scenario} → {duration}")
print(f"\n🏁 Total time: {timedelta(seconds=time.time() - start_all)}")

▶️ Starting: tasmax | ssp245▶️ Starting: tasmin | ssp245▶️ Starting: pr | ssp245


✅ Saved: /content/drive/MyDrive/climate_outputs/wrb_gifs/20kmbuffer/tasmax_ssp245_combined.gif
✅ Done: tasmax | ssp245 in 0:02:11.133967
▶️ Starting: tasmin | ssp585
⏳ 1/6 done | Est. time left: 0:10:55.991185
✅ Saved: /content/drive/MyDrive/climate_outputs/wrb_gifs/20kmbuffer/pr_ssp245_combined.gif
✅ Done: pr | ssp245 in 0:02:12.667637
▶️ Starting: tasmax | ssp585
✅ Saved: /content/drive/MyDrive/climate_outputs/wrb_gifs/20kmbuffer/tasmin_ssp245_combined.gif
✅ Done: tasmin | ssp245 in 0:02:12.715457
▶️ Starting: pr | ssp585
⏳ 2/6 done | Est. time left: 0:04:25.431499
⏳ 3/6 done | Est. time left: 0:02:12.756778
✅ Saved: /content/drive/MyDrive/climate_outputs/wrb_gifs/20kmbuffer/tasmin_ssp585_combined.gif
✅ Done: tasmin | ssp585 in 0:02:09.385759
⏳ 4/6 done | Est. time left: 0:02:10.306733
✅ Saved: /content/drive/MyDrive/climate_outputs/wrb_gifs/20kmbuffer/tasmax_ssp585_combined.gif
✅ Done: tasmax | ssp585